In [1]:
import polars as pl

import pandas as pd
# These commands enlarge the column size of the dataframe so things like 0x... are not truncated
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [2]:
pepe_swaps = pl.read_parquet("pepe_swaps_4.12.23_to_4.19.23.parquet")

In [7]:
pepe_swaps.head(5)

amountOutUSD,amountInUSD,amountOut,amountIn,id,timestamp,account_id,tokenIn_symbol,tokenOut_symbol
f64,f64,f64,f64,str,str,str,str,str
1289.148284,1297.057447,6.4761e17,3.7633e27,"""0xed67bcbcab68…","""1681918703""","""0x41258d0119e4…","""PEPE""","""WETH"""
3162.241888,3184.043386,1.5886e18,9.2382e27,"""0xe3b6d3dfbcb0…","""1681918703""","""0xcbcc2b2ecd19…","""PEPE""","""WETH"""
215.972745,218.968305,6.2662e26,1.1000e17,"""0x86db55cc9daf…","""1681918703""","""0x59abf65b6252…","""WETH""","""PEPE"""
491.370852,497.655238,1.4257e27,2.5000e17,"""0x7a8d1cccf083…","""1681918703""","""0x1aa2a7d52c6f…","""WETH""","""PEPE"""
78.531009,79.624838,2.2785e26,4.0000e16,"""0x4bb4139e03bc…","""1681918703""","""0x268d7e4fca20…","""WETH""","""PEPE"""


In [11]:
# filter by tokenIn_symbol = PEPE
pepe_sells = pepe_swaps.filter(pl.col("tokenIn_symbol") == "PEPE")
pepe_buys = pepe_swaps.filter(pl.col("tokenOut_symbol") == "PEPE")

In [16]:
agg_sells = pepe_sells.groupby("account_id").agg(pl.sum("amountOutUSD").alias("sell_usd")).sort("sell_usd", descending=True)
agg_buys = pepe_buys.groupby("account_id").agg(pl.sum("amountOutUSD").alias("buy_usd")).sort("buy_usd", descending=True)

In [27]:
# polars join on account_id
totals = agg_sells.join(agg_buys, on="account_id", how="left").fill_null(0)

# calculate total profits with polars dataframe
totals = totals.with_columns((pl.col("sell_usd") - pl.col("buy_usd")).alias("profit_usd"))

In [30]:
totals_pd_df = totals.sort("profit_usd", descending=True).to_pandas()

In [35]:
totals_pd_df.head(20)

,account_id,sell_usd,buy_usd,profit_usd
0,0x7607709f3771502bb1fbe3aff84323369144eacf,427254.652659,0.000000,427254.652659
1,0xb58555fcba6479fced7de1485eb054943a09af7b,756956.667538,544889.822254,212066.845284
2,0x044e29b53579e93a0e0609ae6e6f86e8326c4c07,205043.104470,0.000000,205043.104470
3,0xe94e7c652b2701a87b2f73d2213399db5fc611d3,187161.239914,0.000000,187161.239914
4,0x7fe920d2366960abc38b6ab9556b7c15f75f5d35,165637.758420,0.000000,165637.758420
5,0xf27874f53038855afda1ff6049b000e598f900cb,337049.739968,172781.118069,164268.621899
6,0xcb9fbdbd60009b911ad95e35b77257ae558a14a8,147188.440817,0.000000,147188.440817
7,0x758e8229dd38cf11fa9e7c0d5f790b4ca16b3b16,241323.858883,104765.606721,136558.252162
8,0xd850632c9b314a0e6736f45e4ed677d8c0ccb3fb,135596.458291,0.000000,135596.458291
9,0x19848274b125f4f75c6d0524f5196473c023d21a,120844.076116,0.000000,120844.076116
